In [2]:
# 引入基本資料處理用函式庫
import numpy as np
import pandas as pd
import os
import warnings
import random

# 引入 Pytorch 函式庫, 神經網路函式庫, Optimizer優化器 Loss function是要幫助我們判斷誤差值的，而Optimizer是要調整參數，來使Loss越小越好。
import torch 
from torch import nn
import torch.optim as optim

# 資料集分割器, 供多重驗證模型使用
from sklearn.model_selection import StratifiedKFold

# 引入單字,單詞分割器
import tokenizers
# 引入主要模型, RoBERTa (Robustly optimized BERT approach)
from transformers import RobertaModel, RobertaConfig

warnings.filterwarnings('ignore')

此區塊主要用於調整所有用到的函式庫使用同一個種子碼，
確保程式及訓練過程及結果可以重現。確保亂數的值固定

In [3]:
def seed_everything(seed_value):
    #調整 random, numpy, pytorch, python本體 的種子碼
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    # 若有 GPU 版本 Pytorch 可使用
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

# 設定種子碼為 30
seed = 30
seed_everything(seed)